In [1]:
# Clone your GitHub repo (you’ll be prompted to authorize if it's private)
!git clone https://github.com/colterwood/LHL-final-final-project.git

Cloning into 'LHL-final-final-project'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 91 (delta 43), reused 20 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (91/91), 1.14 MiB | 1.06 MiB/s, done.
Resolving deltas: 100% (43/43), done.


In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from io import StringIO
import numpy as np

In [6]:
# List of team codes for 2024 season
teams = ['ATL', 'CHI', 'CON', 'DAL', 'IND', 'LAS', 'MIN', 'NYL', 'PHO', 'SEA', 'WAS', 'LVA']

# Base URL pattern for advanced game logs
base_url = "https://www.basketball-reference.com/wnba/teams/{team}/2024/gamelog-advanced/"

frames = []

for team in teams:
    url = base_url.format(team=team)
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    table = soup.find("table", id="wnba_tgl_advanced")

    if table is not None:
        df_team = pd.read_html(StringIO(str(table)))[0]

        # Combine multi-level headers
        df_team.columns = [f"{a}_{b}" for a, b in df_team.columns]

        # Add team column as first column
        df_team.insert(0, "Team", team)

        frames.append(df_team)
    else:
        print(f"No advanced table found for {team}")

# Combine into single DataFrame
df = pd.concat(frames, ignore_index=True)

In [7]:
df.head()

,Team,Unnamed: 0_level_0_Rk,Unnamed: 1_level_0_Date,Unnamed: 2_level_0_Unnamed: 2_level_1,Unnamed: 3_level_0_Opp,Unnamed: 4_level_0_W/L,Unnamed: 5_level_0_Tm,Unnamed: 6_level_0_Opp,Unnamed: 7_level_0_Unnamed: 7_level_1,Advanced_ORtg,...,Unnamed: 18_level_0_Unnamed: 18_level_1,Offensive Four Factors_eFG%,Offensive Four Factors_TOV%,Offensive Four Factors_ORB%,Offensive Four Factors_FT/FGA,Unnamed: 23_level_0_Unnamed: 23_level_1,Defensive Four Factors_eFG%,Defensive Four Factors_TOV%,Defensive Four Factors_DRB%,Defensive Four Factors_FT/FGA
0,ATL,1,2024-05-15,@,LAS,W,92,81,NaN,114.0,...,NaN,.567,14.7,20.6,.239,NaN,.472,11.9,77.1,.181
1,ATL,2,2024-05-18,@,PHO,L,85,88,NaN,103.6,...,NaN,.485,14.3,18.4,.318,NaN,.455,13.0,78.8,.424
2,ATL,3,2024-05-21,NaN,DAL,W,83,78,NaN,107.0,...,NaN,.465,12.8,35.1,.222,NaN,.443,15.1,67.6,.229
3,ATL,4,2024-05-26,NaN,MIN,L,79,92,NaN,107.0,...,NaN,.500,11.9,27.8,.162,NaN,.605,12.2,81.5,.274
4,ATL,5,2024-05-29,@,WAS,W,73,67,NaN,92.0,...,NaN,.468,19.3,18.5,.242,NaN,.405,14.4,77.5,.095


In [10]:
# Drop unnecessary columns
drop_cols = [
    'Unnamed: 7_level_0_Unnamed: 7_level_1',
    'Unnamed: 18_level_0_Unnamed: 18_level_1',
    'Unnamed: 23_level_0_Unnamed: 23_level_1'
]
df = df.drop(columns=drop_cols, errors='ignore')

# Rename key columns
df = df.rename(columns={
    'Unnamed: 0_level_0_Rk': 'Rk',
    'Unnamed: 1_level_0_Date': 'Date',
    'Unnamed: 3_level_0_Opp': 'Opp',
    'Unnamed: 4_level_0_W/L': 'W/L',
    'Unnamed: 5_level_0_Tm': 'Team_Score',
    'Unnamed: 6_level_0_Opp': 'Opp_Score'
})

# rename the messy column name to 'home_away'
df = df.rename(columns={'Unnamed: 2_level_0_Unnamed: 2_level_1': 'home_away'})

# convert values: '@' becomes 2 (away), everything else becomes 1 (home)
df['home_away'] = df['home_away'].apply(lambda x: 2 if x == '@' else 1)

In [12]:
df.head()

,Team,Rk,Date,home_away,Opp,W/L,Team_Score,Opp_Score,Advanced_ORtg,Advanced_DRtg,...,Advanced_STL%,Advanced_BLK%,Offensive Four Factors_eFG%,Offensive Four Factors_TOV%,Offensive Four Factors_ORB%,Offensive Four Factors_FT/FGA,Defensive Four Factors_eFG%,Defensive Four Factors_TOV%,Defensive Four Factors_DRB%,Defensive Four Factors_FT/FGA
0,ATL,1,2024-05-15,1,LAS,W,92,81,114.0,100.4,...,9.9,22.0,.567,14.7,20.6,.239,.472,11.9,77.1,.181
1,ATL,2,2024-05-18,1,PHO,L,85,88,103.6,107.3,...,9.8,11.6,.485,14.3,18.4,.318,.455,13.0,78.8,.424
2,ATL,3,2024-05-21,1,DAL,W,83,78,107.0,100.5,...,11.6,5.6,.465,12.8,35.1,.222,.443,15.1,67.6,.229
3,ATL,4,2024-05-26,1,MIN,L,79,92,107.0,124.6,...,12.2,3.0,.500,11.9,27.8,.162,.605,12.2,81.5,.274
4,ATL,5,2024-05-29,1,WAS,W,73,67,92.0,84.4,...,11.3,4.3,.468,19.3,18.5,.242,.405,14.4,77.5,.095


In [15]:
df.shape

(504, 26)

In [14]:
# Print each column name on its own line
for col in df.columns:
    print(col)

Team
Rk
Date
home_away
Opp
W/L
Team_Score
Opp_Score
Advanced_ORtg
Advanced_DRtg
Advanced_Pace
Advanced_FTr
Advanced_3PAr
Advanced_TS%
Advanced_TRB%
Advanced_AST%
Advanced_STL%
Advanced_BLK%
Offensive Four Factors_eFG%
Offensive Four Factors_TOV%
Offensive Four Factors_ORB%
Offensive Four Factors_FT/FGA
Defensive Four Factors_eFG%
Defensive Four Factors_TOV%
Defensive Four Factors_DRB%
Defensive Four Factors_FT/FGA


In [ ]:
# Save to data folder
df.to_csv(r"C:\Users\colte\LHL-final-project\LHL-final-final-project\data\2024_basketball_reference_gamelog-advanced.csv", index=False)

In [16]:
# make all column headers lowercase
df.columns = df.columns.str.lower()

In [17]:
# replace '%' with '_pct' in column names
df.columns = df.columns.str.replace('%', '_pct', regex=False)

In [18]:
# rename w/l to win_loss
df = df.rename(columns={'w/l': 'win_loss'})

In [19]:
# convert values: W becomes 1 (win), everything else becomes 2 (loss)
df['win_loss'] = df['win_loss'].apply(lambda x: 1 if x == 'W' else 2)

In [21]:
# print each column with its dtype
for col in df.columns:
    print(f"- {col}: {df[col].dtype}")

- team: object
- rk: object
- date: object
- home_away: int64
- opp: object
- win_loss: int64
- team_score: object
- opp_score: object
- advanced_ortg: object
- advanced_drtg: object
- advanced_pace: object
- advanced_ftr: object
- advanced_3par: object
- advanced_ts_pct: object
- advanced_trb_pct: object
- advanced_ast_pct: object
- advanced_stl_pct: object
- advanced_blk_pct: object
- offensive four factors_efg_pct: object
- offensive four factors_tov_pct: object
- offensive four factors_orb_pct: object
- offensive four factors_ft/fga: object
- defensive four factors_efg_pct: object
- defensive four factors_tov_pct: object
- defensive four factors_drb_pct: object
- defensive four factors_ft/fga: object


In [23]:
# show columns with > 0 null values
df.isnull().sum()[df.isnull().sum() > 0]

,0
rk,12
date,12
opp,12
team_score,12
opp_score,12


In [24]:
# drop rows where 'g#' is null
df = df[df['date'].notnull()]

In [26]:
# show columns with > 0 null values
df.isnull().sum()[df.isnull().sum() > 0]

,0


In [27]:
# show rows where 'date' column is literally 'Date'
df[df['date'] == 'Date']

,team,rk,date,home_away,opp,win_loss,team_score,opp_score,advanced_ortg,advanced_drtg,...,advanced_stl_pct,advanced_blk_pct,offensive four factors_efg_pct,offensive four factors_tov_pct,offensive four factors_orb_pct,offensive four factors_ft/fga,defensive four factors_efg_pct,defensive four factors_tov_pct,defensive four factors_drb_pct,defensive four factors_ft/fga
21,ATL,Rk,Date,1,Opp,2,Tm,Opp,ORtg,DRtg,...,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA,eFG%,TOV%,DRB%,FT/FGA
63,CHI,Rk,Date,1,Opp,2,Tm,Opp,ORtg,DRtg,...,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA,eFG%,TOV%,DRB%,FT/FGA
105,CON,Rk,Date,1,Opp,2,Tm,Opp,ORtg,DRtg,...,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA,eFG%,TOV%,DRB%,FT/FGA
147,DAL,Rk,Date,1,Opp,2,Tm,Opp,ORtg,DRtg,...,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA,eFG%,TOV%,DRB%,FT/FGA
189,IND,Rk,Date,1,Opp,2,Tm,Opp,ORtg,DRtg,...,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA,eFG%,TOV%,DRB%,FT/FGA
231,LAS,Rk,Date,1,Opp,2,Tm,Opp,ORtg,DRtg,...,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA,eFG%,TOV%,DRB%,FT/FGA
273,MIN,Rk,Date,1,Opp,2,Tm,Opp,ORtg,DRtg,...,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA,eFG%,TOV%,DRB%,FT/FGA
315,NYL,Rk,Date,1,Opp,2,Tm,Opp,ORtg,DRtg,...,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA,eFG%,TOV%,DRB%,FT/FGA
357,PHO,Rk,Date,1,Opp,2,Tm,Opp,ORtg,DRtg,...,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA,eFG%,TOV%,DRB%,FT/FGA
399,SEA,Rk,Date,1,Opp,2,Tm,Opp,ORtg,DRtg,...,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA,eFG%,TOV%,DRB%,FT/FGA


In [28]:
# drop rows where 'date' is the string 'Date'
df = df[df['date'] != 'Date']

In [29]:
# convert to datetime (if not already)
df['date'] = pd.to_datetime(df['date'])

# split into year, month, day
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

In [30]:
# drop 'date' column
df = df.drop(columns=['date'])

# build new column order without year/month/day
cols = [col for col in df.columns if col not in ['year', 'month', 'day']]

# find where to insert
insert_at = cols.index('home_away')

# insert year/month/day before 'age'
new_order = cols[:insert_at] + ['year', 'month', 'day'] + cols[insert_at:]

# reorder
df = df[new_order]

In [31]:
df.head()

,team,rk,year,month,day,home_away,opp,win_loss,team_score,opp_score,...,advanced_stl_pct,advanced_blk_pct,offensive four factors_efg_pct,offensive four factors_tov_pct,offensive four factors_orb_pct,offensive four factors_ft/fga,defensive four factors_efg_pct,defensive four factors_tov_pct,defensive four factors_drb_pct,defensive four factors_ft/fga
0,ATL,1,2024,5,15,1,LAS,1,92,81,...,9.9,22.0,.567,14.7,20.6,.239,.472,11.9,77.1,.181
1,ATL,2,2024,5,18,1,PHO,2,85,88,...,9.8,11.6,.485,14.3,18.4,.318,.455,13.0,78.8,.424
2,ATL,3,2024,5,21,1,DAL,1,83,78,...,11.6,5.6,.465,12.8,35.1,.222,.443,15.1,67.6,.229
3,ATL,4,2024,5,26,1,MIN,2,79,92,...,12.2,3.0,.500,11.9,27.8,.162,.605,12.2,81.5,.274
4,ATL,5,2024,5,29,1,WAS,1,73,67,...,11.3,4.3,.468,19.3,18.5,.242,.405,14.4,77.5,.095


In [32]:
# drop the 'rk' column
df = df.drop(columns=['rk'])

In [33]:
# print each column with its dtype
for col in df.columns:
    print(f"- {col}: {df[col].dtype}")

- team: object
- year: int32
- month: int32
- day: int32
- home_away: int64
- opp: object
- win_loss: int64
- team_score: object
- opp_score: object
- advanced_ortg: object
- advanced_drtg: object
- advanced_pace: object
- advanced_ftr: object
- advanced_3par: object
- advanced_ts_pct: object
- advanced_trb_pct: object
- advanced_ast_pct: object
- advanced_stl_pct: object
- advanced_blk_pct: object
- offensive four factors_efg_pct: object
- offensive four factors_tov_pct: object
- offensive four factors_orb_pct: object
- offensive four factors_ft/fga: object
- defensive four factors_efg_pct: object
- defensive four factors_tov_pct: object
- defensive four factors_drb_pct: object
- defensive four factors_ft/fga: object


In [34]:
# reset index and drop the old index
df = df.reset_index(drop=True)

In [35]:
df.head()

,team,year,month,day,home_away,opp,win_loss,team_score,opp_score,advanced_ortg,...,advanced_stl_pct,advanced_blk_pct,offensive four factors_efg_pct,offensive four factors_tov_pct,offensive four factors_orb_pct,offensive four factors_ft/fga,defensive four factors_efg_pct,defensive four factors_tov_pct,defensive four factors_drb_pct,defensive four factors_ft/fga
0,ATL,2024,5,15,1,LAS,1,92,81,114.0,...,9.9,22.0,.567,14.7,20.6,.239,.472,11.9,77.1,.181
1,ATL,2024,5,18,1,PHO,2,85,88,103.6,...,9.8,11.6,.485,14.3,18.4,.318,.455,13.0,78.8,.424
2,ATL,2024,5,21,1,DAL,1,83,78,107.0,...,11.6,5.6,.465,12.8,35.1,.222,.443,15.1,67.6,.229
3,ATL,2024,5,26,1,MIN,2,79,92,107.0,...,12.2,3.0,.500,11.9,27.8,.162,.605,12.2,81.5,.274
4,ATL,2024,5,29,1,WAS,1,73,67,92.0,...,11.3,4.3,.468,19.3,18.5,.242,.405,14.4,77.5,.095


In [36]:
# show columns with > 0 null values
df.isnull().sum()[df.isnull().sum() > 0]

,0


In [38]:
# Replace '/' with _per_
df.columns = df.columns.str.replace('/', '_per_', regex=False)

# Replace ' ' with '_'
df.columns = df.columns.str.replace(' ', '_', regex=False)

In [39]:
# print each column with its dtype
for col in df.columns:
    print(f"- {col}: {df[col].dtype}")

- team: object
- year: int32
- month: int32
- day: int32
- home_away: int64
- opp: object
- win_loss: int64
- team_score: object
- opp_score: object
- advanced_ortg: object
- advanced_drtg: object
- advanced_pace: object
- advanced_ftr: object
- advanced_3par: object
- advanced_ts_pct: object
- advanced_trb_pct: object
- advanced_ast_pct: object
- advanced_stl_pct: object
- advanced_blk_pct: object
- offensive_four_factors_efg_pct: object
- offensive_four_factors_tov_pct: object
- offensive_four_factors_orb_pct: object
- offensive_four_factors_ft_per_fga: object
- defensive_four_factors_efg_pct: object
- defensive_four_factors_tov_pct: object
- defensive_four_factors_drb_pct: object
- defensive_four_factors_ft_per_fga: object


In [40]:
df.to_csv("2024_basketball_reference_gamelog-advanced.csv", index=False)

from google.colab import files

files.download("2024_basketball_reference_gamelog-advanced.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [41]:
# list of columns that should be numeric
cols_to_convert = [col for col in df.columns if col not in ['team', 'opp']]

# convert all non-string columns to numeric, coercing errors
df[cols_to_convert] = df[cols_to_convert].apply(pd.to_numeric, errors='coerce')

In [42]:
# show columns with > 0 null values
df.isnull().sum()[df.isnull().sum() > 0]

,0


In [43]:
# print each column with its dtype in a readable list
for col in df.columns:
    print(f"- {col}: {df[col].dtype}")

- team: object
- year: int32
- month: int32
- day: int32
- home_away: int64
- opp: object
- win_loss: int64
- team_score: int64
- opp_score: int64
- advanced_ortg: float64
- advanced_drtg: float64
- advanced_pace: float64
- advanced_ftr: float64
- advanced_3par: float64
- advanced_ts_pct: float64
- advanced_trb_pct: float64
- advanced_ast_pct: float64
- advanced_stl_pct: float64
- advanced_blk_pct: float64
- offensive_four_factors_efg_pct: float64
- offensive_four_factors_tov_pct: float64
- offensive_four_factors_orb_pct: float64
- offensive_four_factors_ft_per_fga: float64
- defensive_four_factors_efg_pct: float64
- defensive_four_factors_tov_pct: float64
- defensive_four_factors_drb_pct: float64
- defensive_four_factors_ft_per_fga: float64


In [44]:
df.to_csv("2024_basketball_reference_gamelog-advanced.csv", index=False)

from google.colab import files

files.download("2024_basketball_reference_gamelog-advanced.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>